In [1]:
from numpy import *
import pandas as pd
from pyecharts import *

In [2]:
file = r'D:\CS\python for data\chapter8\demo\data\data.xls'
df = pd.read_excel(file)
del df['病程阶段']
del df['转移部位']
del df['确诊后几年发现转移']
df[:5]

,肝气郁结证型系数,热毒蕴结证型系数,冲任失调证型系数,气血两虚证型系数,脾胃虚弱证型系数,肝肾阴虚证型系数,TNM分期
0,0.056,0.460,0.281,0.352,0.119,0.350,H4
1,0.488,0.099,0.283,0.333,0.116,0.293,H4
2,0.107,0.008,0.204,0.150,0.032,0.159,H4
3,0.322,0.208,0.305,0.130,0.184,0.317,H4
4,0.242,0.280,0.131,0.210,0.191,0.351,H4


In [3]:
ranges = [0.179, 0.258, 0.35, 0.504]
numbers = []
for i in range(4):
    rows = df[df['肝气郁结证型系数'] > ranges[i]]
    numbers.append(len(rows))

scatter_data = pd.DataFrame({'肝气郁结证型系数范围': [0.179, 0.258, 0.35, 0.504],
                            '范围内样本个数': numbers},
                            index = ['A1','A2','A3','A4']
                            )
scatter_data

,肝气郁结证型系数范围,范围内样本个数
A1,0.179,686
A2,0.258,331
A3,0.350,53
A4,0.504,0


In [4]:
from sklearn.cluster import KMeans
Kmodel = KMeans(n_clusters=4, n_jobs=4)
Kmodel.fit(df[['热毒蕴结证型系数']].as_matrix())
new_df = df['热毒蕴结证型系数'].copy()
temp = pd.DataFrame(Kmodel.labels_) 
new_df = pd.concat([new_df, temp], axis=1)
new_df.rename(columns={0:'等级'}, inplace=True)
#new_df.columns = ['肝气郁结证型系数', '等级']
Kmodel.cluster_centers_

G:\study\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  app.launch_new_instance()


array([[0.59310345],
       [0.09745614],
       [0.20962895],
       [0.38680447]])

In [5]:
#-*- coding: utf-8 -*-
'''
聚类离散化，最后的result的格式为：
      1           2           3           4
A     0    0.178698    0.257724    0.351843
An  240  356.000000  281.000000   53.000000
即(0, 0.178698]有240个，(0.178698, 0.257724]有356个，依此类推。
'''
from __future__ import print_function
import pandas as pd
from sklearn.cluster import KMeans #导入K均值聚类算法

datafile = r'D:\CS\python for data\chapter8\demo\data\data.xls' #待聚类的数据文件
typelabel ={u'肝气郁结证型系数':'A', u'热毒蕴结证型系数':'B', u'冲任失调证型系数':'C', u'气血两虚证型系数':'D', u'脾胃虚弱证型系数':'E', u'肝肾阴虚证型系数':'F'}
k = 4 #需要进行的聚类类别数

#读取数据并进行聚类分析
data = pd.read_excel(datafile) #读取数据
keys = list(typelabel.keys())
result = pd.DataFrame()

if __name__ == '__main__': #判断是否主窗口运行，如果是将代码保存为.py后运行，则需要这句，如果直接复制到命令窗口运行，则不需要这句。
    for i in range(len(keys)):
        #调用k-means算法，进行聚类离散化
        print(u'正在进行“%s”的聚类...' % keys[i])
        kmodel = KMeans(n_clusters = k, n_jobs = 4) #n_jobs是并行数，一般等于CPU数较好
        kmodel.fit(data[[keys[i]]].as_matrix()) #训练模型

        r1 = pd.DataFrame(kmodel.cluster_centers_, columns = [typelabel[keys[i]]]) #聚类中心
        r2 = pd.Series(kmodel.labels_).value_counts().sort_index()  #分类统计
        r2 = pd.DataFrame(r2, columns = [typelabel[keys[i]]+'n']) #转为DataFrame，记录各个类别的数目
        r = pd.concat([r1, r2], axis = 1).sort_values(by= typelabel[keys[i]])   #匹配聚类中心和类别数目
        r.index = [1, 2, 3, 4]

        r[typelabel[keys[i]]] = r[typelabel[keys[i]]].rolling(2).mean() #rolling_mean()用来计算相邻2列的均值，以此作为边界点。
        r[typelabel[keys[i]]][1] = 0.0 #这两句代码将原来的聚类中心改为边界点。

        result = result.append(r.T)
        
        result = result.sort_index() #以Index排序，即以A,B,C,D,E,F顺序排
    
    print(result)

正在进行“肝肾阴虚证型系数”的聚类...


G:\study\Anaconda3\lib\site-packages\ipykernel\__main__.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
G:\study\Anaconda3\lib\site-packages\ipykernel\__main__.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


正在进行“气血两虚证型系数”的聚类...
正在进行“肝气郁结证型系数”的聚类...
正在进行“热毒蕴结证型系数”的聚类...
正在进行“冲任失调证型系数”的聚类...
正在进行“脾胃虚弱证型系数”的聚类...
        1           2           3           4
A     0.0    0.178698    0.257724    0.351843
An  240.0  356.000000  281.000000   53.000000
B     0.0    0.147818    0.284702    0.450267
Bn  316.0  391.000000  169.000000   54.000000
C     0.0    0.202149    0.289061    0.423537
Cn  297.0  394.000000  204.000000   35.000000
D     0.0    0.172049    0.251583    0.359353
Dn  283.0  375.000000  228.000000   44.000000
E     0.0    0.153398    0.258200    0.376062
En  278.0  314.000000  245.000000   93.000000
F     0.0    0.179143    0.261386    0.354643
Fn  200.0  237.000000  265.000000  228.000000
